In [2]:
PROJECT_ROOT = "../"
import sys
# The following line adds the parent directory to the python path.
sys.path.insert(0, PROJECT_ROOT) 

In [3]:
from importlib import reload
import src.experiment.sample_utils as su
import src.experiment.experiment_utils as exp
import src.dataset_handling.isolet.write_isolet as wi
import src.experiment.dataset_experiments.mnist_experiments as m_exp
import src.experiment.experiment_cpu as exp_cpu

reload(exp_cpu)
reload(su)
reload(exp)
reload(wi)
reload(m_exp)

<module 'src.experiment.dataset_experiments.mnist_experiments' from '/home/hphi344/Documents/GS-DBSCAN-Analysis/analysis_notebooks/../src/experiment/dataset_experiments/mnist_experiments.py'>

In [11]:
k = 5
m = 200
minPts = 50
eps = 0.12

parquet_name = f"isolet_{k}_{m}_{eps}_{minPts}"
dtype = "f32"

isolet_data = f"/home/hphi344/Documents/GS-DBSCAN-Analysis/data/isolet/isolet_data_{dtype}.bin"
isolet_labels = f"/home/hphi344/Documents/GS-DBSCAN-Analysis/data/isolet/isolet_labels_{dtype}.bin"

In [4]:
params = exp.RunParams(datasetFilename=isolet_data, labels_filename=isolet_labels, n=wi.ISOLET_N, d=wi.ISOLET_DIM, D=1024, minPts=minPts, k=k, m=m, eps=eps, alpha=1.2,
                        distancesBatchSize=100, distanceMetric="COSINE",
                        clusterBlockSize=256, clusterOnCpu=True, needToNormalize=True, print_cmd=True,
                        verbose=False, useBatchDbscan=True, timeIt=True, useBatchABMatrices=True,
                        useBatchNorm=True,
                        datasetDType=dtype, ABatchSize=10_000, BBatchSize=28, miniBatchSize=10_000, normBatchSize=10_000, ignoreAdjListSymmetry=False)

exp.run_complete_sdbscan_pipeline(params, parquet_name)

Running GS-DBSCAN

/home/hphi344/Documents/GS-DBSCAN-CPP/build-release/GS-DBSCAN --datasetFilename /home/hphi344/Documents/GS-DBSCAN-Analysis/data/isolet/isolet_data_f16.bin --outputFilename /home/hphi344/Documents/GS-DBSCAN-Analysis/results/results__isolet_data_f16_n7797_d617_D1024_mp50_k5_m200_e0.1200.json --n 7797 --d 617 --D 1024 --minPts 50 --k 5 --m 200 --eps 0.12 --alpha 1.2 --distancesBatchSize 100 --distanceMetric COSINE --clusterBlockSize 256 --datasetDType f16 --miniBatchSize 10000 --ABatchSize 10000 --BBatchSize 28 --normBatchSize 10000 --sigmaEmbed 1 --clusterOnCpu --needToNormalize --useBatchClustering --useBatchABMatrices --useBatchNorm --timeIt
Standard Output:
 Running GsDBSCAN-CPP
Params: 

## PARAMS ##

Data Filename: /home/hphi344/Documents/GS-DBSCAN-Analysis/data/isolet/isolet_data_f16.bin
Output Filename: /home/hphi344/Documents/GS-DBSCAN-Analysis/results/results__isolet_data_f16_n7797_d617_D1024_mp50_k5_m200_e0.1200.json
n: 7797
d: 617
D: 1024
minPts: 50
k: 5
m: 

,args,clusterLabels,numClusters,times,params,nmi
0,\n\n## PARAMS ##\n\nData Filename: /home/hphi3...,"[-1, -1, 0, 0, -1, -1, 0, 0, 0, -1, -1, -1, 0,...",5,"{'adjList': 948, 'constructABMatrices': 269893...","{'d': 617, 'minPts': 50, 'alpha': 1.2, 'distan...",0.264818


In [16]:
d = wi.ISOLET_DIM
eps = 0.12
minPts = 50
k = 5
m = 200
numThreads = 64 # For testing, set to 64 for full run
dist = "Cosine"

run_params_cpu = exp_cpu.CpuRunParams(d, eps, minPts, 1024, k, m, numThreads, dist, datasetFilename=isolet_data, verbose=True,
                            output_file=f"/home/hphi344/Documents/GS-DBSCAN-Analysis/results/isolet/isolet_cpu_k{k}_m{m}_nt{numThreads}_eps{eps}", file_type="bin", labels_filename=isolet_labels,
                            labels_file_type="bin", interval_sampling=0, n=wi.ISOLET_N)

In [42]:
import time

start = time.time()
print(exp_cpu.run_cpu_sdbscan(run_params_cpu)["times"].iloc[0])
print(f"Time: {time.time() - start}")

eps: 0.12
minPts: 50
n_points: 7797
n_features: 617
n_proj: 1024
topK: 5
topM: 200
distance: Cosine
cluster noise: 0
kernel features: 617
sigma: 2600
interval sampling: 0
sDbscan-1NN prob: 0
n_threads: 64
Check X supporting distance time = 4[ms]
Build index time = 19[ms]
Number of core points: 680
Find core points time = 78[ms]
Clustering noisy point time = 0[ms]
Number of clusters: 7
Form clusters time  (including clustering noise) = 0[ms]
Writing times to file: /home/hphi344/Documents/GS-DBSCAN-Analysis/results/isolet/isolet_cpu_k5_m200_nt64_eps0.12_times.txt
Times map written to file successfully.
NMI: 0.27681021777697407
{'buildIndexTime': 19000000.0, 'checkXSupportingDistanceTime': 4000000.0, 'findCorePointsTime': 78000000.0, 'formClustersTime': 0.0, 'overall': 101000000.0}
Time: 0.11363720893859863
